In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import pandas as pd

Preprocessing Function for words

In [0]:
from __future__ import division
from __future__ import print_function

import random
import numpy as np
import cv2
import imutils
from google.colab.patches import cv2_imshow

def preprocess(img, imgSize, dataAugmentation=False):
	"put img into target img of size imgSize, transpose for TF and normalize gray-values"

	# there are damaged files in IAM dataset - just use black image instead
	if img is None:
		img = np.zeros([imgSize[1], imgSize[0]])
		print('test')
	img = cv2.resize(img, imgSize)
	img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	# imgBlurred = cv2.GaussianBlur(img, (5,5), 0)                        # blur

  # #                                                       # filter image from grayscale to black and white
	# imgThresh = cv2.adaptiveThreshold(img,                           # input image
  #                                     255,                                  # make pixels that pass the threshold full white
  #                                     cv2.ADAPTIVE_THRESH_GAUSSIAN_C,       # use gaussian rather than mean, seems to give better results
  #                                     cv2.THRESH_BINARY_INV,                # invert so foreground will be white, background will be black
  #                                     11,                                   # size of a pixel neighborhood used to calculate threshold value
  #                                     2)                                    # constant subtracted from the mean or weighted mean

  #   # cv2.imshow("imgThresh", imgThresh)
	cv2_imshow(img)  
	# # return imgThresh

In [0]:
img = cv2.imread('/content/drive/My Drive/APM Project/Data/Characters only/Img/Sample029/img029-003.png', cv2.IMREAD_UNCHANGED) #change the path here
imgSize = (43, 32)
test = cv2.resize(img, imgSize, interpolation = cv2.INTER_AREA)
# cv2_imshow(img)

error: ignored

In [0]:
image_labels = pd.read_csv('/content/drive/My Drive/APM Project/Data/Characters only/Label file.csv',header = None)
image_labels.columns = ['Image Path','Label','Mapping']

imgSize = (43, 32)
character_data = pd.DataFrame(columns = ['Character_Image', 'Character_Label','Mapped_Label'])

for i in range(len(image_labels)):
  img = preprocess(np.uint8(cv2.imread('/content/drive/My Drive/APM Project/Data/Characters only/'+image_labels['Image Path'][i])), imgSize)
  character_data= character_data.append({'Character_Image':img, 'Character_Label':image_labels['Label'][i], 'Mapped_Label':np.int64(image_labels['Mapping'][i])}, ignore_index=True)

In [0]:
from sklearn.model_selection import train_test_split
temp = np.array(character_data['Character_Image'].to_list())
X_train, X_test, y_train, y_test = train_test_split(temp, character_data['Mapped_Label'], test_size=0.10, random_state=42)

In [0]:
# Now we prepare train_data and test_data after resizing to 32X43.
X_train = X_train.reshape(-1,1376).astype(np.float32)
X_test = X_test.reshape(-1,1376).astype(np.float32) 
# Create labels for train and test data
y_train = y_train.to_numpy(dtype='int64')[:,np.newaxis]
y_test = y_test.to_numpy(dtype = 'int64')[:,np.newaxis]

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)

In [0]:
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [0]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [0]:
X_train = pd.DataFrame(data=X_train[0:,0:], index=[i for i in range(X_train.shape[0])], columns=['f'+str(i) for i in range(X_train.shape[1])]) 

In [0]:
X_test = pd.DataFrame(data=X_test[0:,0:], index=[i for i in range(X_test.shape[0])], columns=['f'+str(i) for i in range(X_test.shape[1])]) 

In [0]:
# # Initiate kNN, train the data, then test it with test data for k=1
# knn = cv2.ml.KNearest_create()
# knn.train(X_train, cv2.ml.ROW_SAMPLE, y_train)
# ret, result, neighbours, dist = knn.findNearest(X_test, k=3)

# # Now we check the accuracy of classification
# # For that, compare the result with test_labels and check which are wrong
# matches = result==y_test
# correct = np.count_nonzero(matches)
# accuracy = correct*100.0/result.size
# print (accuracy)


In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 3)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
acc = knn_model.score(X_test, y_test)
print('Accuracy: {:.2f}%'.format(acc*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 36.95%


In [0]:
temp = knn_model.predict(X_test)

In [0]:
temp[0]

53

In [0]:
cv2_imshow(X_train[0])

IndexError: ignored

In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=1000)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.36656891495601174

In [0]:
##############################################################################################################################################

In [0]:
from skimage.feature import hog
def feature_extraction(image):
    return hog(color.rgb2gray(image), orientations=8, pixels_per_cell=(10, 10), cells_per_block=(5, 5))
def predict(df):
    predict = knn.predict(df.reshape(1,-1))[0]
    predict_proba = knn.predict_proba(df.reshape(1,-1))
    return predict, predict_proba[0][predict]
digits = []
# load your image from file
# extract featuress
hogs = list(map(lambda x: feature_extraction(x), digits))
# apply k-NN model created in previous
predictions = list(map(lambda x: predict(x), hogs))

In [0]:
feature_extraction('/content/drive/My Drive/APM Project/Data/Characters only/Img/Sample029/img029-027.png')

NameError: ignored

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from imutils import paths
import argparse
knn_model = KNeighborsClassifier(n_neighbors = 3)
knn_model.fit(X_train, y_train)
acc = knn_model.score(X_test, y_test)
print('Accuracy: {:.2f}%'.format(acc*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


Accuracy: 34.90%


Loading IAM Dataset images to implement model on

In [0]:
from __future__ import division
from __future__ import print_function

import os
import random
import numpy as np
import cv2
##from SamplePreprocessor import preprocess


class Sample:
	"sample from the dataset"
	def __init__(self, gtText, filePath):
		self.gtText = gtText
		self.filePath = filePath


class Batch:
	"batch containing images and ground truth texts"
	def __init__(self, gtTexts, imgs):
		self.imgs = np.stack(imgs, axis=0)
		self.gtTexts = gtTexts


class DataLoader:
	"loads data which corresponds to IAM format, see: http://www.fki.inf.unibe.ch/databases/iam-handwriting-database" 

	def __init__(self, filePath, batchSize, imgSize, maxTextLen):
		"loader for dataset at given location, preprocess images and text according to parameters"

		assert filePath[-1]=='/'

		self.dataAugmentation = False
		self.currIdx = 0
		self.batchSize = batchSize
		self.imgSize = imgSize
		self.samples = []
	
		f=open(filePath+'words.txt')
		chars = set()
		bad_samples = []
		bad_samples_reference = ['a01-117-05-02.png', 'r06-022-03-05.png']
		for line in f:
			# ignore comment line
			if not line or line[0]=='#':
				continue
			
			lineSplit = line.strip().split(' ')
			assert len(lineSplit) >= 9
			
			# filename: part1-part2-part3 --> part1/part1-part2/part1-part2-part3.png
			fileNameSplit = lineSplit[0].split('-')
			fileName = filePath + 'words/' + fileNameSplit[0] + '/' + fileNameSplit[0] + '-' + fileNameSplit[1] + '/' + lineSplit[0] + '.png'

			# GT text are columns starting at 9
			gtText = self.truncateLabel(' '.join(lineSplit[8:]), maxTextLen)
			chars = chars.union(set(list(gtText)))

			# check if image is not empty
			##if not os.path.getsize(fileName):
			#	bad_samples.append(lineSplit[0] + '.png')
			#	continue

			# put sample into list
			self.samples.append(Sample(gtText, fileName))

		# some images in the IAM dataset are known to be damaged, don't show warning for them
		if set(bad_samples) != set(bad_samples_reference):
			print("Warning, damaged images found:", bad_samples)
			print("Damaged images expected:", bad_samples_reference)

		# split into training and validation set: 95% - 5%
		splitIdx = int(0.95 * len(self.samples))
		self.trainSamples = self.samples[:splitIdx]
		self.validationSamples = self.samples[splitIdx:]

		# put words into lists
		self.trainWords = [x.gtText for x in self.trainSamples]
		self.validationWords = [x.gtText for x in self.validationSamples]

		# number of randomly chosen samples per epoch for training 
		self.numTrainSamplesPerEpoch = 25000 
		
		# start with train set
		self.trainSet()

		# list of all chars in dataset
		self.charList = sorted(list(chars))


	def truncateLabel(self, text, maxTextLen):
		# ctc_loss can't compute loss if it cannot find a mapping between text label and input 
		# labels. Repeat letters cost double because of the blank symbol needing to be inserted.
		# If a too-long label is provided, ctc_loss returns an infinite gradient
		cost = 0
		for i in range(len(text)):
			if i != 0 and text[i] == text[i-1]:
				cost += 2
			else:
				cost += 1
			if cost > maxTextLen:
				return text[:i]
		return text


	def trainSet(self):
		"switch to randomly chosen subset of training set"
		self.dataAugmentation = True
		self.currIdx = 0
		random.shuffle(self.trainSamples)
		self.samples = self.trainSamples[:self.numTrainSamplesPerEpoch]

	
	def validationSet(self):
		"switch to validation set"
		self.dataAugmentation = False
		self.currIdx = 0
		self.samples = self.validationSamples


	def getIteratorInfo(self):
		"current batch index and overall number of batches"
		return (self.currIdx // self.batchSize + 1, len(self.samples) // self.batchSize)


	def hasNext(self):
		"iterator"
		#return self.currIdx + self.batchSize <= len(self.samples)
		return self.currIdx + self.batchSize <= 10
		
		
	def getNext(self):
		"iterator"
		batchRange = range(self.currIdx, self.currIdx + self.batchSize)
		gtTexts = [self.samples[i].gtText for i in batchRange]
		imgs = [preprocess(cv2.imread(self.samples[i].filePath), self.imgSize, self.dataAugmentation) for i in batchRange]
		self.currIdx += self.batchSize
		return Batch(gtTexts, imgs)





In [0]:
fnTrain = '/content/drive/My Drive/APM Project/Data/testing/'
imgSize = (128, 32)
maxTextLen = 32
loader = DataLoader(fnTrain, 1, imgSize, maxTextLen)

Warning, damaged images found: []
Damaged images expected: ['a01-117-05-02.png', 'r06-022-03-05.png']


In [0]:
# while loader.hasNext():
#   batch = loader.getNext() 


In [0]:
#for testing
for i in range(5):
  print(loader.samples[i].filePath)

/content/drive/My Drive/APM Project/Data/testing/words/a03/a03-063/a03-063-06-06.png
/content/drive/My Drive/APM Project/Data/testing/words/e07/e07-012/e07-012-02-02.png
/content/drive/My Drive/APM Project/Data/testing/words/p03/p03-121/p03-121-06-04.png
/content/drive/My Drive/APM Project/Data/testing/words/c03/c03-094b/c03-094b-09-01.png
/content/drive/My Drive/APM Project/Data/testing/words/m04/m04-061/m04-061-03-12.png


In [0]:
##  HARD CODING VALUES ONLY FOR TESTING PURPOSES. PLEASE REMOVE THIS WHEN THE ENTIRE DATASET IS LOADED

fnTrain = '/content/drive/My Drive/APM Project/Data/testing/'
imgSize = (128, 32)
maxTextLen = 32
loader = DataLoader(fnTrain, 1, imgSize, maxTextLen)

loader.samples[0].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/e06/e06-015/e06-015-08-03.png'
loader.samples[1].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/f01/f01-066/f01-066-00-02.png'
loader.samples[2].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/g01/g01-067/g01-067-03-04.png'
loader.samples[3].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/h01/h01-024/h01-024-03-05.png'
loader.samples[4].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/e04/e04-008/e04-008-00-09.png'
loader.samples[5].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/a05/a05-089/a05-089-04-03.png'
loader.samples[6].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/b05/b05-017/b05-017-03-05.png'
loader.samples[7].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/b06/b06-059/b06-059-05-05.png'
loader.samples[8].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/c03/c03-021d/c03-021d-00-01.png'
loader.samples[9].filePath = '/content/drive/My Drive/APM Project/Data/testing/words/d01/d01-056/d01-056-07-01.png'

loader.samples[0].gtText = 'before'
loader.samples[1].gtText = 'you'
loader.samples[2].gtText = 'read'
loader.samples[3].gtText = 'business'
loader.samples[4].gtText = 'pinned'
loader.samples[5].gtText = 'I'
loader.samples[6].gtText = 'for'
loader.samples[7].gtText = 'u'
loader.samples[8].gtText = 'Fanny'
loader.samples[9].gtText = 'been'

Warning, damaged images found: []
Damaged images expected: ['a01-117-05-02.png', 'r06-022-03-05.png']


In [0]:
x = []
y = []
while loader.hasNext():
  batch = loader.getNext() 
  y.extend(batch.gtTexts)
  x.extend(batch.imgs)

Now creating the sliding window

In [0]:
#Finding the countours
cells = []
letter_pointer_df = pd.DataFrame(columns = ['Image', 'intX','intY','intW','intH']) # Dataframe to store green bounding box pixel positions

for i in range(len(x)):
  imgInput = x[i]
  # cv2_imshow(imgInput)
  if imgInput is None:
    print ('test')
  # imgGray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY) # Convert to grayscale since it's easier to detect bright objects against dark background
  newRet, binaryThreshold = cv2.threshold(imgInput,127,255,cv2.THRESH_BINARY_INV) #inverting black and white regions, making the letters white
  rectkernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,10)) # Build a kernel to scan over the image
  rectdilation = cv2.dilate(binaryThreshold, rectkernel, iterations = 1) #Dilate the image values to create good enough contrast
  outputImage = imgInput.copy() 
  # cv2_imshow(outputImage)
  rand_var, npaContours, npaHierarchy = cv2.findContours(rectdilation.copy(),        
                                              cv2.RETR_EXTERNAL,                 
                                              cv2.CHAIN_APPROX_SIMPLE)  #Creating contours to find position of the word in the image

  for npaContour in npaContours:                  
      if cv2.contourArea(npaContour) > 120:       #Change this value   #Filtering on minimum contour area (need to optimize this)

          [intX, intY, intW, intH] = cv2.boundingRect(npaContour)        #Find Red bounding box (outer box for word)

          cv2.rectangle(outputImage,          #For display purposes, drawing red rectangle on the image 
                (intX, intY),                 # upper left corner
                (intX+intW,intY+intH),        # lower right corner
                (0, 0, 255),                  # red
                2)                            # thickness

          # Get subimage of word and find contours of that word
          imgROI = binaryThreshold[intY:intY+intH, intX:intX+intW]   


          rand_var2, subContours, subHierarchy = cv2.findContours(imgROI.copy(),     #Finding countours for individual characters   
                                              cv2.RETR_EXTERNAL,                 
                                              cv2.CHAIN_APPROX_SIMPLE) 
          letter_pointer_df = letter_pointer_df.append({'Image': imgInput.copy(), 'intX':intX, 'intY': intY, 'intW' : intW, 'intH' : intH}, ignore_index=True) #Storing all bounding boxes 
  #cells.append(gray)
  #cv2_imshow(outputImage)
  

In [0]:
#Input mapping file
mapping_file = pd.read_csv('/content/drive/My Drive/APM Project/Data/Characters only/Labels to actual character file.csv',header = None)
mapping_file.columns = ['Mapping','Label']

map_dict = mapping_file['Label'].to_dict()

In [0]:
map_dict[20]

'K'

In [0]:
sliding_window_size = (43,32)
#Retreiving characters from formatted word image using bounding box
predicted_labels = []
for i in range(len(letter_pointer_df)):
  Image,intX,intY,intW,intH = letter_pointer_df.iloc[i]
  imgCopy =Image.copy()
  original_intX = intX
  sliding_window_img_list = []
  label_string = ''
  while (intX+sliding_window_size[0] <= original_intX + intW):
    sliding_window_img = imgCopy[intY:intY+sliding_window_size[1], intX:intX+sliding_window_size[0]]
    sliding_window_img_list.append(sliding_window_img)
    intX = intX + 20
  sliding_window_img_list = (np.asarray(sliding_window_img_list)).reshape(-1,1376).astype(np.float32) 
  # sliding_window_img_list = np.asarray(sliding_window_img_list,dtype = 'int64')[:,np.newaxis]
  ret, result, neighbours, dist = knn.findNearest(sliding_window_img_list, k=3)
  for j in range(len(result)):
    label_string = label_string + map_dict[int(result[j])] 
  predicted_labels.append(label_string)
 # cv2_imshow(x)

In [0]:
predicted_labels

['aaiic',
 '8BTTT',
 'ieIig',
 'wrjua',
 'a4d0c',
 'yNOH4',
 'eH4Fw',
 'cdCLe',
 'cJT41',
 'j1a1I']

In [0]:
# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result==y_test
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print (accuracy)

1